In [ ]:
import torch 
import numpy as np 
import pandas as pd
import os 
import torch.nn 
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import random
import torchaudio
import matplotlib.pyplot as plt
import torch.nn as nn 
from tqdm.auto import tqdm
%reload_ext autoreload
%autoreload 2
%cd ..

In [ ]:
!unzip final_ttp_audio.zip

In [ ]:
!upzip final_tts_audio.zip

In [ ]:
class TextProcessor():
  def __init__(self, filtered_txt_paths, each_lyric, sorted_txt_paths, teach_forcing_prob=0.8):
    self.filtered_txt_paths = filtered_txt_paths
    self.each_lyric = each_lyric
    self.sorted_txt_paths = sorted_txt_paths
    self.teach_forcing_prob = teach_forcing_prob

  def read_text(self, txt_path):
    with open(txt_path, 'r') as f:
      text = f.read()
    return text
  
  def list_read_text(self, txt_path_list):
    all_txt = []
    #print(txt_path_list)
    for i in txt_path_list:
      pth = self.sorted_txt_paths[i]
      with open(pth, 'r') as f:
        text = f.read()
        all_txt.append(text)
        
    return '\n '.join(all_txt) + '\n'
      

  def get_random_sentence(self, idx_num, random_mode=False):
    #if teach_forcing == True:
    
    if random_mode == False:

      key_with_value = [key for key, value in self.each_lyric.items() if value == (idx_num)]
      if len(key_with_value) > 50:
        key_with_value = random.sample(key_with_value, 50)
        return key_with_value
      
      
    else:
      key_with_value = [random.randint(0, len(self.filtered_txt_paths)) for _ in range(30)]

    return key_with_value
  
  def random_position(self, text_list):
    random.shuffle(text_list)
    return text_list
  
  def __len__(self):
    return len(self.each_lyric)

  def __getitem__(self, idx):
    txt_path = self.filtered_txt_paths[idx]
    idx_num = self.each_lyric[int(txt_path.stem.split('lyric')[1])] # 몇번 곡인지 반환 
    input_text = self.read_text(txt_path)
    
    around_sentence = self.get_random_sentence(idx_num)
    around_sentence = self.random_position(around_sentence)
    around_sentence =  self.list_read_text(around_sentence)
    

    if random.random() < self.teach_forcing_prob: # True 
      if isinstance(idx_num, tuple):
        rand_idx = random.randint(0, 1)
        idx_num = idx_num[rand_idx]
        around_sentence = self.get_random_sentence(idx_num, False)
        around_sentence = self.random_position(around_sentence)
        around_sentence = self.list_read_text(around_sentence) 
        
      else:
        around_sentence = self.get_random_sentence(idx_num, False)
        around_sentence = self.random_position(around_sentence)
        around_sentence =  self.list_read_text(around_sentence)

    else: 
      around_sentence = self.get_random_sentence(idx_num, True)
      around_sentence = self.random_position(around_sentence)
      around_sentence =  self.list_read_text(around_sentence)


    return input_text, around_sentence

In [ ]:
def read_text(self, txt_path):
    with open(txt_path, 'r') as f:
      text = f.read()
    return text


In [ ]:
import random
a = [1,2,3,4,5]
random.shuffle(a)
print(a)

In [1]:
from pathlib import Path
import random 
import pickle
class TextProcessor():
  def __init__(self, id_list, lyric_path, each_lyric, random_ratio=0.8):
    self.lyric_path = Path(lyric_path)
    self.filtered_id = id_list
    self.lyrics_list = [lyric_path+i+'.txt' for i in self.filtered_id]
    self.each_lyric_list = self.read_text(each_lyric).split('\n')
    self.random_ratio = random_ratio
    # self.teach_forcing_prob = teach_forcing_prob
  
  def read_text(self, txt_path):
    with open(txt_path, 'r') as f:
      text = f.read()
    return text
  
  def get_all_sentence_list(self):
    all_sentence = []
    for i in self.lyrics_list:
      all_sentence.append(self.read_text(i))
    return list(set(all_sentence))
  
  def __len__(self):
    return len(self.lyrics_list)
  
  def get_random_sentence(self, lyric_name, random_mode=False):
    # lyric_name = 'lyric0-1.txt'
    if random.random() < self.random_ratio:
      random_mode = True
    if random_mode == False:
      lyric_index = int(lyric_name.split('-')[0][5:])
      lyrics = self.each_lyric_list[lyric_index]
      lyrics = lyrics.split(', ')
      if len(lyrics) > 30:
        lyrics = random.sample(lyrics, 30)
      # print(lyrics)
      random.shuffle(lyrics)
      
    else: 
      lyrics = random.sample(self.get_all_sentence_list(), 30)
      
    return lyrics
  
  def __getitem__(self, idx):
    txt_path = self.lyrics_list[idx]
    lyric_name = txt_path.split('/')[-1][:-4]
    # print(lyric_name)
    input_text = self.read_text(txt_path)
    random.seed(0)
    output_sentence = self.get_random_sentence(lyric_name)
    output_sentence = '\n'.join(output_sentence)
    return input_text, output_sentence, lyric_name
  


lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'
each_lyric = '/home/daewoong/userdata/danbi/each_song_lyrics.txt'
filtered_id_list = pickle.load(open('/home/daewoong/userdata/danbi/thirty_second_filtered_id.pkl', 'rb'))

processor = TextProcessor(filtered_id_list, lyric_path, each_lyric)
print(processor[1302])

('무슨 사주가 기박하여 세상살이 살고 보니', '놈산 시상 내 못살고\n한두달에 못하다가\n마른자리 느그 입헤\n열달에 배 슬어서\n어매 어매 우리 어매\n신세 신세 내신세야\n나난시에 남도 낳건마는\n세상살이 보소\n겉만 타먼 놈이 안디\n죽자니는 청춘이고\n나타난것 무엇이냐\n속만 타먼 놈이 아냐\n세상이 그렇구나\n뭣을 묵고 나를 나서\n삼식이식 기렀건만\n육지 백판 이내수고 어디가고\n내가슴에 불은\n놈난시에 나도 낳고\n진자리는 내가 눕고\n아들딸을 낳건마는\n나도 세상 살어날때\n세상살이 살고 보니\n신세한탄 한이 없네\n살자니는 고생이로고나\n자식들아 느그 날때\n무슨 사주가 기박하여\n우리 어매 울아부지 날 날적에\n애야 서라 내 팔자야\n마당갓에 모닥불만', 'lyric893-4')


In [ ]:
len(processor)

In [ ]:
from pathlib import Path
import torchaudio
def filter_length(audio_path, max_len=30):
  audio_list = [str(i) for i in audio_path.rglob('*.wav')]
  filtered_audio_list = []
  for i in audio_list:
    audio, sr = torchaudio.load(i)
    if len(audio)/sr <= float(max_len):
      filtered_audio_list.append(i)
  return filtered_audio_list
audio_path = Path('/home/daewoong/userdata/danbi/final_tts_audio')
filtered_audio_list = filter_length(audio_path)
print(len(filtered_audio_list))

In [ ]:
lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'
each_lyric = '/home/daewoong/userdata/danbi/each_song_lyrics.txt'
filtered_id_list = pickle.load(open('/home/daewoong/userdata/danbi/thirty_second_filtered_id.pkl', 'rb'))

processor = TextProcessor(filtered_id_list, lyric_path, each_lyric)

In [ ]:
# %cd ..
# with open('thirty_second_filtered_id.pkl', 'wb') as f:
#   pickle.dump(filtered_id_list, f)

# Mel spec으로 바꿔놓기

In [17]:
import pickle

with open('/home/daewoong/userdata/danbi/thirty_second_filtered_id.pkl', 'rb') as f:
  filtered_id_list = pickle.load(f)
filtered_id_list[:10]

['lyric3696-1',
 'lyric5730-4',
 'lyric7157-9',
 'lyric4166-0',
 'lyric5710-4',
 'lyric5855-6',
 'lyric4112-28',
 'lyric2841-2',
 'lyric5429-2',
 'lyric5472-25']

from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="ko", task="transcribe", predict_timestamps=True)


audio = self.processor.feature_extractor(audio[0] , sampling_rate=sr, padding_value = 0.0, return_tensors="pt", return_attention_mask = True)

In [1]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="ko", task="transcribe", predict_timestamps=True)


In [ ]:
import torchaudio
import torch
audio_path = '/home/daewoong/userdata/danbi/final_tts_audio'
save_path = '/home/daewoong/userdata/danbi/encoder_result'
lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'

for audio_name in filtered_id_list:
  audio, sr = torchaudio.load(f'{audio_path}/{audio_name}.wav')
  spec = processor.feature_extractor(audio.mean(dim=0) , sampling_rate=sr, padding_value = 0.0, return_tensors="pt", return_attention_mask = True)
  torch.save(spec, f'{save_path}/{audio_name}.pt')

In [ ]:
class AudioEncoder(nn.Module):
    def __init__(
        self, n_mels: int, n_ctx: int, n_state: int, n_head: int, n_layer: int
    ):
        super().__init__()
        self.conv1 = Conv1d(n_mels, n_state, kernel_size=3, padding=1)
        self.conv2 = Conv1d(n_state, n_state, kernel_size=3, stride=2, padding=1)
        self.register_buffer("positional_embedding", sinusoids(n_ctx, n_state))

        self.blocks: Iterable[ResidualAttentionBlock] = nn.ModuleList(
            [ResidualAttentionBlock(n_state, n_head) for _ in range(n_layer)]
        )
        self.ln_post = LayerNorm(n_state)

    def forward(self, x: Tensor):
        """
        x : torch.Tensor, shape = (batch_size, n_mels, n_ctx)
            the mel spectrogram of the audio
        """
        x = F.gelu(self.conv1(x))
        x = F.gelu(self.conv2(x))
        x = x.permute(0, 2, 1) # (batch_size, n_ctx, n_state)

        assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"
        x = (x + self.positional_embedding).to(x.dtype)

        for block in self.blocks:
            x = block(x)

        x = self.ln_post(x)
        return x

In [22]:

audio_path = '/home/daewoong/userdata/danbi/final_tts_audio'
save_path = '/home/daewoong/userdata/danbi/encoder_result'
lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'
each_lyric = '/home/daewoong/userdata/danbi/each_song_lyrics.txt'

from data_utils import MinyoDataset, custom_collate_fn
from torch.utils.data import Dataset, DataLoader, random_split
import whisper
device = 'cuda'
dataset = MinyoDataset(audio_path, lyric_path, processor, filtered_id_list, each_lyric, max_len = 1024)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn, num_workers=8, pin_memory=True)
model = whisper.load_model("/home/daewoong/userdata/danbi/whisper_pretrain/large-v2.pt", device='cpu')
model.to(device)


In [28]:
device = 'cuda'
model.to(device)
batch = next(iter(dataloader))
model.encoder(batch[0].to(device))

tensor([[[-4.5030e-01, -2.1314e-01, -7.1459e-02,  ..., -4.2323e+00,
          -8.2900e-02, -6.4309e-01],
         [ 8.5316e-02,  4.7627e-01,  1.7633e-01,  ..., -3.9504e+00,
           4.6285e-01, -1.1680e+00],
         [ 6.0381e-02,  6.4522e-01, -2.2624e-01,  ..., -3.2380e+00,
           1.1161e+00, -1.0413e+00],
         ...,
         [-8.2729e-03, -4.5520e-03, -1.1558e-02,  ...,  6.3755e-04,
          -1.1148e-02, -2.2260e-03],
         [ 2.7415e-01,  5.4565e-02, -5.2823e-01,  ...,  1.3546e-02,
           3.7851e-01, -6.3340e-02],
         [ 5.0610e-01, -1.7489e-01, -3.6949e-01,  ..., -2.1853e-01,
           5.1372e-01,  2.2935e-01]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward0>)

In [40]:

audio_path = '/home/daewoong/userdata/danbi/final_tts_audio'
save_path = '/home/daewoong/userdata/danbi/encoder_result'
lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'
each_lyric = '/home/daewoong/userdata/danbi/each_song_lyrics.txt'

from tqdm import tqdm
model.eval()
with torch.inference_mode():
  for i, batch in tqdm(enumerate(dataloader)):
    encoder_value = model.encoder(batch[0].to(device))
    encoder_value = encoder_value.squeeze(0).to('cpu')
    audio_name = filtered_id_list[i]
    torch.save(encoder_value, f'{save_path}/{audio_name}.pt')
    break
encoder_value

0it [00:01, ?it/s]


tensor([[-4.5030e-01, -2.1314e-01, -7.1459e-02,  ..., -4.2323e+00,
         -8.2900e-02, -6.4309e-01],
        [ 8.5316e-02,  4.7627e-01,  1.7633e-01,  ..., -3.9504e+00,
          4.6285e-01, -1.1680e+00],
        [ 6.0381e-02,  6.4522e-01, -2.2624e-01,  ..., -3.2380e+00,
          1.1161e+00, -1.0413e+00],
        ...,
        [-8.2729e-03, -4.5520e-03, -1.1558e-02,  ...,  6.3755e-04,
         -1.1148e-02, -2.2260e-03],
        [ 2.7415e-01,  5.4565e-02, -5.2823e-01,  ...,  1.3546e-02,
          3.7851e-01, -6.3340e-02],
        [ 5.0610e-01, -1.7489e-01, -3.6949e-01,  ..., -2.1853e-01,
          5.1372e-01,  2.2935e-01]])

In [32]:
encoder_value.squeeze(0).shape

torch.Size([1500, 1280])

In [ ]:
with torch.inference_mode():
  model.encoder.eval()

In [1]:
class MinyoDataset():
  def __init__(self, result_path, lyric_path, processor, filtered_id_list, each_lyric, max_len):#filtered_audio_paths, processor, filtered_txt_paths, each_lyric, sorted_txt_paths, max_len = 1024):
    self.encoder_result_paths = Path(result_path)
    self.filtered_id_list = filtered_id_list
    self.text_process = TextProcessor(filtered_id_list, lyric_path, each_lyric)
    self.processor = processor
    self.max_len = max_len
  
  def trunaction(self, text, token_text):
    words = text.split()
    
    while len(token_text['input_ids'][0]) > self.max_len:
      words = words[:-1]
      text = '\n '.join(words) + '\n'
      token_text = self.processor.tokenizer(text, return_tensors="pt")
    return token_text

  def around_pad_seqence(self, token_text):
    
    if len(token_text['input_ids'][0]) < self.max_len:
      pad_length = self.max_len - len(token_text['input_ids'][0])
      
      padding = torch.full((pad_length,), 50257, dtype=torch.long)
      attn_padding = torch.full((pad_length,), 0, dtype=torch.long)
      
      around_text_ids = torch.cat([token_text['input_ids'][0], padding])
      around_text_mask = torch.cat([token_text['attention_mask'][0], attn_padding])    
      return around_text_ids, around_text_mask

    else: 
      return token_text['input_ids'][0], token_text['attention_mask'][0]

  def __len__(self):
    return len(self.text_process)
  
  def __getitem__(self, idx):
    input_text, around_text, lyric_name = self.text_process[idx]
    around_text_org = around_text
    audio_encoder_value = torch.load(self.encoder_result_paths / (lyric_name+'.pt'))
    # audio, sr = torchaudio.load(self.audio_paths / (lyric_name+'.wav'))
    # audio = self.processor.feature_extractor(audio[0] , sampling_rate=sr, padding_value = 0.0, return_tensors="pt", return_attention_mask = True)
    input_text = self.processor.tokenizer(input_text, return_tensors="pt")
    around_text = self.processor.tokenizer(around_text, return_tensors="pt")
    
    num_tokens = len(around_text['input_ids'][0])
    
    if num_tokens > self.max_len:
      around_text = self.trunaction(around_text_org, around_text)

    around_text_ids, around_text_mask = self.around_pad_seqence(around_text)
    return audio_encoder_value, input_text.input_ids[0], input_text.attention_mask[0], around_text_ids, around_text_mask

In [3]:
!nvidia-smi

Sun Jun 11 13:57:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:C1:00.0 Off |                  Off |
| 55%   82C    P2   260W / 300W |  33388MiB / 49140MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from pathlib import Path
encoder_result = Path('/home/daewoong/userdata/danbi/encoder_result')


'/home/daewoong/userdata/danbi/encoder_result/lyric3653-4.pt'

In [21]:
import torch
emcoder_result = str(list(encoder_result.rglob('*.pt'))[4])
emcoder_result
result = torch.load(emcoder_result)
print(result.shape)
if result[0].shape == torch.Size([1280]):
  print('hi')

torch.Size([1500, 1280])
hi
